In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd
from datetime import datetime, timezone, timedelta

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store


In [2]:
# Use fixed datetime since April data is not available
fixed_hour = pd.Timestamp("2025-01-30 12:00:00", tz="UTC")
print(f"📅 Using fixed hour for predictions: {fixed_hour}")


📅 Using fixed hour for predictions: 2025-01-30 12:00:00+00:00


In [3]:
# Connect to Hopsworks feature store
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

# Read all data and filter for the fixed hour
df = fg.read()
predictions = df[df["pickup_hour"] >= fixed_hour]
predictions.head()


2025-05-10 13:32:28,536 INFO: Initializing external client
2025-05-10 13:32:28,537 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 13:32:30,828 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.71s) 


,pickup_location_id,predicted_demand,pickup_hour
0,HB105,2.0,2025-05-01 00:00:00+00:00
1,JC115,2.0,2025-05-01 00:00:00+00:00
2,HB101,3.0,2025-05-01 00:00:00+00:00
3,JC066,5.0,2025-05-01 00:00:00+00:00
4,HB102,5.0,2025-05-01 00:00:00+00:00


In [50]:
# Sort and display top 10 predictions
top_locations = predictions.sort_values("predicted_demand", ascending=False)
top_locations[["pickup_location_id", "pickup_hour", "predicted_demand"]].head(10)


,pickup_location_id,pickup_hour,predicted_demand
3,JC066,2025-05-01 00:00:00+00:00,5.0
4,HB102,2025-05-01 00:00:00+00:00,5.0
2,HB101,2025-05-01 00:00:00+00:00,3.0
0,HB105,2025-05-01 00:00:00+00:00,2.0
1,JC115,2025-05-01 00:00:00+00:00,2.0
5,JC009,2025-05-01 00:00:00+00:00,0.0


In [30]:
######################

In [51]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-05-10 18:00:00+00:00


In [ ]:
fixed_hour = pd.Timestamp("2025-02-30 12:00:00", tz="UTC")

In [52]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= fixed_hour)

    return query.read()

In [53]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-10 13:20:22,636 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 13:20:22,645 INFO: Initializing external client
2025-05-10 13:20:22,646 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 13:20:23,815 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


In [56]:
query.info()

AttributeError: 'Query' object has no attribute 'info'. 

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  5 non-null      object                 
 1   predicted_demand    5 non-null      float64                
 2   pickup_hour         5 non-null      datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), object(1)
memory usage: 252.0+ bytes


In [13]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-05-10 11:50:49,365 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 11:50:49,374 INFO: Initializing external client
2025-05-10 11:50:49,374 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 11:50:50,612 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Current UTC time: 2025-05-10 15:50:49.365404+00:00
Next hour: 2025-05-10 16:00:00+00:00
Querying for date range: 2025-05-10 to 2025-05-11
Filtering for hour: 2025-05-10 16:00
Found 0 records


In [14]:
now = datetime.now(timezone.utc)

In [15]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [16]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [17]:
dt = current_date.ceil('h')

In [43]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
query = query.filter(fg.pickup_hour >= "2025-01-13 04:00")
# query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-05-10 12:21:17,925 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 12:21:17,933 INFO: Initializing external client
2025-05-10 12:21:17,934 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 12:21:19,171 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.41s) 


In [44]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,HB105,0.0,2025-05-01 00:00:00+00:00
1,JC115,1.0,2025-05-01 00:00:00+00:00
2,HB101,1.0,2025-05-01 00:00:00+00:00
3,JC066,0.0,2025-05-01 00:00:00+00:00
4,HB102,0.0,2025-05-01 00:00:00+00:00


In [45]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.37s) 


,pickup_location_id,predicted_demand,pickup_hour


In [46]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,HB105,0.0,2025-05-01 00:00:00+00:00
1,JC115,1.0,2025-05-01 00:00:00+00:00
2,HB101,1.0,2025-05-01 00:00:00+00:00
3,JC066,0.0,2025-05-01 00:00:00+00:00
4,HB102,0.0,2025-05-01 00:00:00+00:00


In [21]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [22]:
predictions = fetch_next_hour_predictions()

2025-05-10 11:54:11,673 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 11:54:11,680 INFO: Initializing external client
2025-05-10 11:54:11,681 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 11:54:13,061 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Current UTC time: 2025-05-10 15:54:11.673147+00:00
Next hour: 2025-05-10 16:00:00+00:00
Found 0 records


In [23]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)


2025-05-10 11:54:30,168 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 11:54:30,175 INFO: Initializing external client
2025-05-10 11:54:30,177 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 11:54:31,270 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


In [24]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.37s) 


In [25]:
df["pickup_hour"].max()

Timestamp('2025-05-01 00:00:00+0000', tz='Etc/UTC')

In [26]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-05-10 16:00:00+00:00


In [27]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
